In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense

In [ ]:
# Načtení dat
df = pd.read_csv('sorted_test_data.csv', parse_dates=['Date'])

In [ ]:
df.head()

In [ ]:
# Vybrání relevantních sloupců
data = df[['Duration', 'Priority', 'Equipment_ID']]

In [ ]:
# Normalizace dat
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

In [ ]:
scaled_data[:10]

In [ ]:
# Příprava trénovacích dat
X, y = [], []
for i in range(len(data)-60):
    X.append(scaled_data[i:i+60])
    y.append(scaled_data[i+60, 0])

X, y = np.array(X), np.array(y)

In [ ]:
X.shape

In [ ]:
# Rozdělení dat na trénovací a testovací sady
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Definice LSTM modelu
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mse')

In [ ]:
model.summary()

In [ ]:
# Trénování modelu
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

In [ ]:
# Vykreslení loss per epoch
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Trénovací loss')
plt.plot(history.history['val_loss'], label='Validační loss')
plt.title('Hodnota ztráty (Loss) v průběhu epoch')
plt.xlabel('Epochy')
plt.ylabel('Hodnota ztráty')
plt.legend()
plt.show()

In [ ]:
# Predikce na testovacích datech
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(np.concatenate((np.zeros((len(predictions), 2)), predictions), axis=1))[:, 0]

In [ ]:
# Vizualizace predikcí a skutečných hodnot
plt.figure(figsize=(12, 6))
plt.plot(df.index[-len(y_test):], y_test, label='Skutečné hodnoty')
plt.plot(df.index[-len(predictions):], predictions, label='Predikce')
plt.title('Predikce délek trvání poruch pomocí LSTM')
plt.xlabel('Datum')
plt.ylabel('Doba trvání (sekundy)')
plt.legend()
plt.show()